# Key input info

In [2]:
root_name = '250630'
root_dir = rf"../../data/output/mosa/{root_name}"

In [3]:
import os
os.environ['GDAL_DATA'] = r'/usr/share/gdal'
from test4plot2 import plot_double, clearing, plot_cross_tab
from analysis import basic_stat, find_knee
from spatial_plot import plot_compared_cs, plot_zone_cs, match_cs, plot_delta_zone
from delay_analysis import plot_demand_heatmap
import pandas as pd
import geopandas as gpd
from pyproj import CRS
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from IPython.display import IFrame
warnings.simplefilter("ignore")

In [4]:
pd.options.display.float_format = '{:.3f}'.format  # 所有浮点数保留零位小数
means_stat_columns = ['avg_built_cs', 'avg_fast_per_cs', 'avg_slow_per_cs','avg_extra_large','avg_extra_medium','avg_extra_small',
                      'large_model','medium_model','small_model']
cities = pd.read_csv(rf"../../data/18cities.csv")
order_map = {city: idx for idx, city in enumerate(cities["city"])}

MEANS_PARETO = []
KNEES = []
SUPPLE = []
for name in os.listdir(root_dir):
    full_path = os.path.join(root_dir, name)
    if os.path.isdir(full_path) and name.endswith(f'_{root_name}'):
        city_name = name[:-len(f'_{root_name}')]
        print(city_name)
        cs_gdf = gpd.read_file(rf'../../data/input/cs_gdf/{city_name}.shp', crs=CRS.from_epsg(4507))  # 充电站
        cs_gdf = cs_gdf.drop_duplicates(subset=['node_id'], keep='first').reset_index(drop=True)
        cs_num = len(cs_gdf)
        OBJ_G = pd.read_csv(rf"../../data/output/mosa/{root_name}/{city_name}_{root_name}/archive_objs.csv")
        VAR_G = pd.read_csv(rf"../../data/output/mosa/{root_name}/{city_name}_{root_name}/archive_vars.csv")
        CV_G = pd.read_csv(rf"../../data/output/mosa/{root_name}/{city_name}_{root_name}/archive_cvs.csv")
        OBJ_G = clearing(OBJ_G)
        print(f'{len(OBJ_G)}, {len(VAR_G)}')
        STAT_G = basic_stat(OBJ_G, VAR_G, CV_G, cs_num)
        MEANS_G = STAT_G[['avg_built_cs', 'avg_fast_per_cs', 'avg_slow_per_cs','avg_extra_large','avg_extra_medium','avg_extra_small']].mean()
        MODES_G = STAT_G.iloc[:, -3:].mode().iloc[0]
        OVER_G = pd.DataFrame([MEANS_G.tolist() + MODES_G.tolist()], 
                      columns=means_stat_columns)
        # 添加city_name和cs_num列并调整列顺序
        OVER_G['city_name'] = city_name
        OVER_G['candidate_num'] = cs_num
        OVER_G = OVER_G[['city_name', 'candidate_num'] + means_stat_columns]
        MEANS_PARETO.append(OVER_G)

        knee_no_G = find_knee(STAT_G)
        row_G = STAT_G.loc[STAT_G['solution_no'] == knee_no_G]
        row_G['city_name'] = city_name
        row_G['candidate_num'] = cs_num
        KNEES.append(row_G)

        vs_parking_df = pd.read_csv(rf"../../data/input/vs_parking_nodeid//{city_name}.csv")
        vehicle_count = vs_parking_df['v_name'].nunique()
        operational_distance = vs_parking_df['distance'].sum()
        SUPPLE.append([city_name,vehicle_count,operational_distance])
        
        
MEANS_PARETO = pd.concat(MEANS_PARETO, ignore_index=True).sort_values('candidate_num', ascending=False)
KNEES = pd.concat(KNEES, ignore_index=True).sort_values('candidate_num', ascending=False)
KNEES['built_ratio'] = KNEES['avg_built_cs'] / KNEES['candidate_num']
front = ['city_name']
rest = [c for c in KNEES.columns if c not in front]
# 重新排序
KNEES = KNEES[ front + rest ]
SUPPLE = pd.DataFrame(SUPPLE, columns=['city_name','vehicle_count','operational_distance'])

上海市
441, 441
乌鲁木齐市
271, 271
北京市
435, 435
南京市
299, 299
哈尔滨市
335, 335
威海市
228, 228
广州市
405, 405
成都市
363, 363
昆明市
256, 256
武汉市
294, 294
沈阳市
293, 293
深圳市
396, 396
湘潭市
212, 212
珠海市
227, 227
绵阳市
273, 273
莆田市
212, 212
西安市
307, 307
重庆市
384, 384


## 1. Overvall

### 1.1 Average statistics of Pareto solutions

In [7]:
MEANS_PARETO = MEANS_PARETO.reset_index(drop=True)
MEANS_PARETO['avg_charger_ratio'] = MEANS_PARETO['avg_fast_per_cs'] / MEANS_PARETO['avg_slow_per_cs']
MEANS_PARETO["order"] = MEANS_PARETO["city_name"].map(order_map)
MEANS_PARETO = (
    MEANS_PARETO
      .sort_values("order")
      .drop(columns="order")
      .reset_index(drop=True)
)

NameError: name 'KEY_IND' is not defined

In [14]:
MEANS_PARETO

,city_name,candidate_num,avg_built_cs,avg_fast_per_cs,avg_slow_per_cs,avg_extra_large,avg_extra_medium,avg_extra_small,large_model,medium_model,small_model,avg_charger_ratio
0,北京市,1551,775.384,0.134,4.403,1.968,0.000,2.336,4,2,1,0.030
1,上海市,1241,547.905,0.080,4.378,2.388,0.356,0.000,4,2,1,0.018
2,重庆市,1260,627.841,0.081,3.447,1.424,0.042,0.000,4,2,1,0.023
3,成都市,901,388.052,0.089,4.354,0.000,0.000,0.000,4,2,1,0.020
4,广州市,944,436.773,0.075,4.655,6.264,0.681,0.257,4,2,1,0.016
5,深圳市,626,259.654,0.121,6.652,0.018,0.000,0.000,4,2,1,0.018
6,南京市,643,276.134,0.194,4.962,1.324,0.000,0.000,4,2,1,0.039
7,武汉市,701,305.554,0.076,3.465,1.422,0.000,0.000,4,2,1,0.022
8,西安市,451,184.137,0.047,5.345,0.000,0.000,0.000,4,2,1,0.009
9,威海市,189,82.776,0.076,3.098,0.000,0.000,0.000,4,2,1,0.024


### 1.2 Average statistics of knee points

In [16]:
KNEES['built_ratio'] = KNEES['built_ratio'].map('{:.2f}'.format)
KNEES = KNEES.reset_index(drop=True)
KNEES.rename(columns={'obj1': 'knee_cost','obj2': 'knee_emission','avg_built_cs': 'knee_built_cs','avg_fast_per_cs': 'knee_fast_per_cs',
                     'avg_slow_per_cs': 'knee_slow_per_cs','avg_extra_large': 'knee_extra_large','avg_extra_medium': 'knee_extra_medium',
                     'avg_extra_small': 'knee_extra_small', 'built_ratio': 'knee_built_ratio'}, inplace=True)
KNEES['knee_charger_ratio'] = KNEES['knee_fast_per_cs'] / KNEES['knee_slow_per_cs']
KNEES["order"] = KNEES["city_name"].map(order_map)
KNEES = (
    KNEES
      .sort_values("order")
      .drop(columns="order")
      .reset_index(drop=True)
)

In [27]:
KNEES

,city_name,solution_no,knee_cost,knee_emission,knee_built_cs,knee_fast_per_cs,knee_slow_per_cs,knee_extra_large,knee_extra_medium,knee_extra_small,large_model,medium_model,small_model,candidate_num,knee_built_ratio,knee_charger_ratio
0,北京市,SA146903,3413.546,100972.361,664,0.208,4.883,9,0,2,4,2,1,1551,0.43,0.043
1,上海市,SA145287,2131.703,68493.344,457,0.123,4.689,4,1,0,4,2,1,1241,0.37,0.026
2,重庆市,SA139616,2176.059,55047.404,528,0.169,3.771,2,0,0,4,2,1,1260,0.42,0.045
3,成都市,SA145325,1432.195,42829.616,314,0.137,5.092,0,0,0,4,2,1,901,0.35,0.027
4,广州市,SA146159,2110.623,60262.512,382,0.141,5.073,4,0,1,4,2,1,944,0.40,0.028
5,深圳市,SA145865,1396.611,47956.806,193,0.358,8.187,0,0,0,4,2,1,626,0.31,0.044
6,南京市,SA146660,1198.316,33534.446,221,0.348,5.656,2,0,0,4,2,1,643,0.34,0.062
7,武汉市,SA144304,1057.369,33266.681,252,0.091,3.940,2,0,0,4,2,1,701,0.36,0.023
8,西安市,SA143677,772.696,27012.299,125,0.144,6.888,0,0,0,4,2,1,451,0.28,0.021
9,威海市,SA145021,170.684,6485.741,38,0.289,5.763,0,0,0,4,2,1,189,0.20,0.050


### 1.3 Key indicators from knee points

In [21]:
KEY_IND = pd.merge(SUPPLE,KNEES,on='city_name')
KEY_IND['vehicle_count'] = KEY_IND['vehicle_count']+KEY_IND['knee_extra_large']+KEY_IND['knee_extra_medium']+KEY_IND['knee_extra_small']
KEY_IND['knee_cost_per_1Mkm'] = KEY_IND['knee_cost']/KEY_IND['operational_distance'] * 1000000
KEY_IND['knee_emission_per_1Mkm'] = KEY_IND['knee_emission']/KEY_IND['operational_distance'] * 1000000
KEY_IND['knee_cs_per_1Mkm'] = KEY_IND['knee_built_cs']/KEY_IND['operational_distance'] * 1000000
KEY_IND['knee_charger_per_1Mkm'] = (KEY_IND['knee_fast_per_cs']+KEY_IND['knee_slow_per_cs'])*KEY_IND['knee_built_cs']/KEY_IND['operational_distance'] * 1000000
KEY_IND['knee_ev_per_1Mkm'] = KEY_IND['vehicle_count'] / KEY_IND['operational_distance'] * 1000000
KEY_IND = KEY_IND[['city_name','knee_cost_per_1Mkm','knee_emission_per_1Mkm','knee_cs_per_1Mkm','knee_charger_per_1Mkm','knee_ev_per_1Mkm']]
KEY_IND["order"] = KEY_IND["city_name"].map(order_map)
KEY_IND = (
    KEY_IND
      .sort_values("order")
      .drop(columns="order")
      .reset_index(drop=True)
)

In [23]:
KEY_IND

,city_name,cost_per_1Mkm,emission_per_1Mkm,station_per_1Mkm,charger_per_1Mkm,ev_per_1Mkm
0,北京市,2.267,67.068,0.441,2.245,18.482
1,上海市,2.004,64.392,0.430,2.067,16.587
2,重庆市,2.590,65.514,0.628,2.475,22.248
3,成都市,2.145,64.147,0.470,2.459,18.145
4,广州市,2.270,64.826,0.411,2.143,17.641
5,深圳市,1.891,64.928,0.261,2.233,15.431
6,南京市,2.349,65.731,0.433,2.601,18.572
7,武汉市,2.076,65.325,0.495,1.995,15.682
8,西安市,1.868,65.292,0.302,2.125,15.146
9,威海市,1.739,66.085,0.387,2.344,12.971


## 2. Pareto front

In [ ]:
city_name = '北京市'
OBJ_G = pd.read_csv(rf"../../data/output/{root_name}/{city_name}_{root_name}/archive_objs.csv")
OBJ_G = clearing(OBJ_G)
plot_double(OBJ_G,save_pic=False)